In [1]:
import numpy as np
import glob2 as glob
import pandas as pd
import copy
from tqdm import tqdm

In [2]:
files = glob.glob('4_8_model_weights_3_layers/*.npy')

weights_dict = {'bias' : {}, 'weight' : {}}

for file in files:
    layer_name = file.split('/')[-1].split('.')[0]
    if layer_name.split('_')[-1] == 'bias':
        weights_dict['bias'].update({layer_name.split('_')[0]: np.load(file)})
    else:
        weights_dict['weight'].update({layer_name.split('_')[0]: np.load(file)})

test_df = pd.read_csv('Iris.csv').drop(columns = ['Id'])
test_df['Species'] = test_df['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})
test_y = test_df['Species']
test_x = test_df.drop(columns = ['Species'])
order= ['fc1', 'fc2', 'fc3']

In [3]:
def choose_neurons_numeric(chosen_neurons, _weights_dict):
    
    weights_dict_ = copy.deepcopy(_weights_dict)
        
    weights_dict_['weight']['fc1'] = weights_dict_['weight']['fc1'][chosen_neurons[0], :]
    weights_dict_['bias']['fc1'] = weights_dict_['bias']['fc1'][chosen_neurons[0]]

    weights_dict_['weight']['fc2'] = weights_dict_['weight']['fc2'][chosen_neurons[1], :][:, chosen_neurons[0]]
    weights_dict_['bias']['fc2'] = weights_dict_['bias']['fc2'][chosen_neurons[1]]

    weights_dict_['weight']['fc3'] = weights_dict_['weight']['fc3'][: ,chosen_neurons[1]]
    
    return weights_dict_


def choose_neurons_slicing(chosen_neurons, _weights_dict):
    weights_dict_ = copy.deepcopy(_weights_dict)

    #############
    ## LAYER 1 ##
    #############

    # Adding expression to the weights of Layer 1
    print('Adding expression to the weights of Layer 1')
    new_fc = []
    for i in tqdm(range(weights_dict_['weight']['fc1'].shape[0])):
        new_fc_row = []
        for j in range(weights_dict_['weight']['fc1'].shape[1]):
            new_fc_row.append(chosen_neurons[0][i] * weights_dict_['weight']['fc1'][i][j])
        new_fc.append(new_fc_row)

    weights_dict_['weight']['fc1'] = np.array(new_fc)

    # Adding expression to the Bias of Layer 1
    print('Adding expression to the bias of Layer 1')
    new_fc = []
    for i in tqdm(range(weights_dict_['bias']['fc1'].shape[0])):
        new_fc.append(chosen_neurons[0][i] * weights_dict_['bias']['fc1'][i])
    
    weights_dict_['bias']['fc1'] = np.array(new_fc)


    #############
    ## LAYER 2 ##
    #############

    # Adding expression to the weights of Layer 2
    print('Adding expression to the weights of Layer 2')
    
    new_fc = weights_dict_['weight']['fc2'].copy().astype(object)
    for i in tqdm(range(weights_dict_['weight']['fc2'].shape[0])):
        for j in range(weights_dict_['weight']['fc2'].shape[1]):
            new_fc[i][j] *= (chosen_neurons[1][i] * chosen_neurons[0][j])

    weights_dict_['weight']['fc2'] = new_fc

    # Adding expression to the Bias of Layer 2
    print('Adding expression to the bias of Layer 2')
    new_fc = []
    for i in tqdm(range(weights_dict_['bias']['fc2'].shape[0])):
        new_fc.append(chosen_neurons[1][i] * weights_dict_['bias']['fc2'][i])
    
    weights_dict_['bias']['fc2'] = np.array(new_fc)

    #############
    ## LAYER 3 ##
    #############
    
    # Adding expression to the weights of Layer 3
    print('Adding expression to the weights of Layer 3')
    
    new_fc = weights_dict_['weight']['fc3'].copy().astype(object)
    for i in tqdm(range(weights_dict_['weight']['fc3'].shape[0])):
        for j in range(weights_dict_['weight']['fc3'].shape[1]):
            new_fc[i][j] *= chosen_neurons[1][j]

    weights_dict_['weight']['fc3'] = new_fc

    return weights_dict_

def predict_expressions(expression_matrix, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], expression_matrix.T) + weights_dict_['bias']['fc1']
    # x = np.maximum(0, x)

    print('Done 1 layer')

    x = np.dot(weights_dict_['weight']['fc2'], x.T) + weights_dict_['bias']['fc2']
    # x = np.maximum(0, x)


    print('Done 2 layer')

    x = np.dot(weights_dict_['weight']['fc3'], x.T) + weights_dict_['bias']['fc3']
    # x = np.maximum(0, x)

    print('Done 3 layer')

    x /= np.sum(x) # softmax

    return x

def predict_numeric(img_df_row, weights_dict_):

    x = np.dot(weights_dict_['weight']['fc1'], img_df_row.T) + weights_dict_['bias']['fc1']
    x = np.maximum(0, x)

    x = np.dot(weights_dict_['weight']['fc2'], x.T) + weights_dict_['bias']['fc2']
    x = np.maximum(0, x)

    x = np.dot(weights_dict_['weight']['fc3'], x.T) + weights_dict_['bias']['fc3']
    x = np.maximum(0, x)

    x /= np.sum(x) # softmax

    return x

def print_neurons_shape(weights_dict_):
    print('Layer Number\t|\t Weights Shape\t|\t Bias Shape')
    print('-'*60)
    print('Layer 1  \t|\t', weights_dict_['weight']['fc1'].shape, '\t|\t' , weights_dict_['bias']['fc1'].shape)
    print('Layer 2  \t|\t', weights_dict_['weight']['fc2'].shape, '\t|\t' , weights_dict_['bias']['fc2'].shape)
    print('Layer 3  \t|\t', weights_dict_['weight']['fc3'].shape, '\t|\t' , weights_dict_['bias']['fc3'].shape)
    
def cross_entropy_loss(label, y_hat):
    y = np.zeros(10)
    y[label] = 1
    return -np.sum(y * np.log(y_hat))

print_neurons_shape(weights_dict)

Layer Number	|	 Weights Shape	|	 Bias Shape
------------------------------------------------------------
Layer 1  	|	 (8, 4) 	|	 (8,)
Layer 2  	|	 (4, 8) 	|	 (4,)
Layer 3  	|	 (3, 4) 	|	 (3,)


### DoCplex

In [190]:
from docplex.cp.model import CpoModel as Model

m = Model()

X_fc1 = np.array(m.binary_var_list(8, name='Layer 1 Neurons'))
X_fc2 = np.array(m.binary_var_list(4, name='Layer 2 Neurons'))

W_hat_fc1_weight = X_fc1.reshape((-1, 1))  * weights_dict['weight']['fc1']
W_hat_fc1_bias = weights_dict['bias']['fc1'] * X_fc1

W_hat_fc2_weight = X_fc2.reshape((-1, 1)) * weights_dict['weight']['fc2']
W_hat_fc2_weight = W_hat_fc2_weight * X_fc1.reshape((1, -1))
W_hat_fc2_bias = X_fc2 * weights_dict['bias']['fc2']

W_hat_fc3_weight = weights_dict['weight']['fc3'] * X_fc2.reshape((1, -1))

sub_chunk = 50
x_chunk = test_x.iloc[:sub_chunk]
y_chunk = test_y.iloc[:sub_chunk]

out_x = (np.dot(x_chunk, W_hat_fc1_weight.T) + W_hat_fc1_bias)

# out_x = (np.dot(out_x, W_hat_fc2_weight.T) + W_hat_fc2_bias)
# out_x = np.dot(out_x, W_hat_fc3_weight.T) + weights_dict['bias']['fc3']

labels = np.zeros_like(out_x)
for i in range(len(out_x)):
    labels[i, y_chunk[i]] = 1 

losses = []
for i in range(len(out_x)):
    for u in range(len(out_x[i])):
        # if labels[i][u] == 1:
        losses.append(labels[i][u] * m.log(out_x[i][u] + 10e-6))
        
number_of_nuerons_used = X_fc1.sum() + X_fc2.sum()

doodoo = m.sum([np.sum(out_x[i]) for i in range(len(out_x))])

m.minimize(losses[0] + losses[1] + losses[2])

In [191]:
sol = m.solve(log_output=True)

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Minimization problem - 3 variables, 0 constraints
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 1.0 (before), 1.0 (after)
 !  . Memory usage      : 299.7 kB (before), 299.7 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0          3                 -
 + New bound is -11.51293
 *     -11.51293        1  0.02s        1      (gap is 0.00%)
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! Best objective         : -11.51293 (optimal - effective tol. is 0.00115129)
 ! Best bound             : -11.51293


 ! ----------------------------------------------------------------------------
 ! Number of branches     : 23
 ! Number of fails        : 23
 ! Total memory usage     : 2.3 MB (2.3 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.02s (0.02s engine + 0.00s extraction)
 ! Search speed (br. / s) : 2300.0
 ! ----------------------------------------------------------------------------


In [192]:
[print(sol.get_all_var_solutions()[i]) for i in range(len(sol.get_all_var_solutions()))]

Layer 1 Neurons_0=0
Layer 1 Neurons_1=0
Layer 1 Neurons_2=0


[None, None, None]

In [193]:
print(sol.get_objective_value())

-11.512925464970229
